In [3]:
import dotenv
import numpy as np
import os
import random
import mne
import enum


dotenv.load_dotenv()

class PredCtChannelMapping(enum.Enum):
    Fp1 = 1
    Fp2 = 3
    F3 = 8
    F4 = 12
    C3 = 26
    C4 = 30
    P3 = 46
    P4 = 50
    O1 = 61
    O2 = 63
    F7 = 6
    F8 = 14
    T7 = 24 ## was T3
    T8 = 32 ## was T4
    P7 = 44 ## was T5
    P8 = 52 ## was T6

class ModmaChannelMapping(enum.Enum):
    Fp1 = 22
    Fp2 = 9
    F3 = 24
    F4 = 124
    C3 = 36
    C4 = 104
    P3 = 52
    P4 = 92
    O1 = 70
    O2 = 83
    F7 = 33
    F8 = 122
    T3 = 45
    T4 = 108
    T5 = 58
    T6 = 96


modma16ChannelsDirPath = os.getenv("MODMA_16CH_DIR")
predct16ChannelsDirPath = os.getenv("PREDCT_16CH_DIR")

modmaNumpyDirPath = modma16ChannelsDirPath + "/numpyFiles/"
predctNumpyDirPath = predct16ChannelsDirPath + "/numpyFiles/"

# === CHANGE THIS TO SWITCH BETWEEN MODMA AND PREDCT DATASET ===
which16ChannelsDirPath = predct16ChannelsDirPath          # !!!!
whichNumpyDirPath = predctNumpyDirPath                    # !!!!

# --- SET VARIABLES FOR PREPROCESSING AND EPOCHING ---
DATASET_SFREQ = 500         # !!!!
EPOCH_LENGTH = 60  
EPOCH_OVERLAP = 0.5  

# --- WHERE TO SAVE THE PROCESSED DATA ---
toSaveDirPath = which16ChannelsDirPath + f"/epoch{EPOCH_LENGTH}sFiles/"

In [4]:
for fileName in os.listdir(whichNumpyDirPath):
    if fileName.endswith(".npy"):  
        filePath = os.path.join(whichNumpyDirPath, fileName)
        print(f"Processing file: {filePath}")
        eegData = np.load(filePath)

        chNames = [f"Ch{i+1}" for i in range(eegData.shape[0])]  
        chTypes = ["eeg"] * eegData.shape[0] 
        info = mne.create_info(ch_names=chNames, sfreq=DATASET_SFREQ, ch_types=chTypes)
        raw = mne.io.RawArray(eegData, info)
        raw.filter(l_freq=0.5, h_freq=40, fir_design='firwin')
        raw.notch_filter(freqs=50, fir_design='firwin')

        if DATASET_SFREQ == 500:
            print("Downsampling to 250 Hz...")
            raw.resample(sfreq = DATASET_SFREQ / 2)
        else:
            print("No downsampling needed.")

        event_interval = int(raw.info['sfreq'] * EPOCH_LENGTH / 2) 
        events = np.array([[i, 0, 1] for i in range(0, raw.n_times - int(raw.info['sfreq'] * EPOCH_LENGTH), event_interval)])
        eventId = {"Dummy": 1}

        tmin = 0
        tmax = EPOCH_LENGTH - 1 / raw.info['sfreq']  
        print(f"Creating {EPOCH_LENGTH}-second epochs with 50% overlap...")
        epochs = mne.Epochs(raw, events, eventId, tmin=tmin, tmax=tmax, baseline=None, preload=True)
        print(f"Epochs shape (n_epochs, n_channels, n_times): {epochs.get_data().shape}")
        
        os.makedirs(toSaveDirPath, exist_ok=True)
        epoch_file = os.path.join(toSaveDirPath, fileName.replace(".npy", ".fif"))
        epochs.save(epoch_file, overwrite=True)
        print(f"Saved epochs to: {epoch_file}")

Processing file: /Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/numpyFiles/599_HC.npy
Creating RawArray with float64 data, n_channels=16, n_times=90000
    Range : 0 ... 89999 =      0.000 ...   179.998 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:


- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 3301 samples (6.602 s)

Downsampling to 250 Hz...
Creating 60-second epochs with 50% overla

/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12932/1732569282.py:32: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/epoch60sFiles/599_HC.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_file, overwrite=True)
/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12932/1732569282.py:32: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/epoch60sFiles/578_HC.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_file, overwrite=True)


Downsampling to 250 Hz...
Creating 60-second epochs with 50% overlap...
Not setting metadata
4 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 4 events and 15000 original time points ...
0 bad epochs dropped
Epochs shape (n_epochs, n_channels, n_times): (4, 16, 15000)
Saved epochs to: /Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/epoch60sFiles/586_MDD.fif
Processing file: /Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/numpyFiles/517_HC.npy
Creating RawArray with float64 data, n_channels=16, n_times=90000
    Range : 0 ... 89999 =      0.000 ...   179.998 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband

/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12932/1732569282.py:32: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/epoch60sFiles/586_MDD.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_file, overwrite=True)
/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12932/1732569282.py:32: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/epoch60sFiles/517_HC.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_file, overwrite=True)


...

/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12932/1732569282.py:32: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/epoch60sFiles/528_HC.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_file, overwrite=True)
/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12932/1732569282.py:32: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/epoch60sFiles/558_MDD.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_file, overwrite=True)


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 3301 samples (6.602 s)

Downsampling to 250 Hz...
Creating 60-second epochs with 50% overlap...
Not setting metadata
4 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 4 events and 15000 original time points ...
0 bad epochs dropped
Epochs shape (n_epochs, n_channels, n_times): (4, 16, 15000)
Saved epochs to: /Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/epoch60s

/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12932/1732569282.py:32: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/epoch60sFiles/618_MDD.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_file, overwrite=True)
/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12932/1732569282.py:32: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/epoch60sFiles/608_MDD.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_file, overwrite=True)


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 3301 samples (6.602 s)

Downsampling to 250 Hz...
Creating 60-second epochs with 50% overlap...
Not setting metadata
4 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 4 events and 15000 original time points ...
0 bad epochs dropped
Epochs shape (n_epochs, n_channels, n_times): (4, 16, 15000)
Saved epochs to: /Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/epoch60s

/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12932/1732569282.py:32: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/data/dataset/predct/eyeopen_16Channels/epoch60sFiles/627_MDD.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_file, overwrite=True)
